<a href="https://colab.research.google.com/github/KekaiApana/datasci112_final_project/blob/main/DATASCI_112_Supreme_Court_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DATASCI 112 Final Project: Supreme Court Data Analysis**
### *By: Kekai and Adrian*

This project explores the Cornell Supreme Court Oral Arguments Corpus (https://convokit.cornell.edu/documentation/supreme.html). This data includes data from cases spanning 1955 to 2019.

**Research question: Can we predict how Chief Justice Roberts will vote on a case based on their linguistic patterns in oral arguments?**

In this file, we CHANGE THIS DESCRIPTION.

In [ ]:
import pandas as pd
df_scotus = pd.read_csv("scotus_roberts_data (3).csv")
df_scotus

,utt_id,text,speaker,reply_to,meta.case_id,meta.speaker_type,year,title,court,votes.j__john_paul_stevens,...,votes_side.j__david_h_souter,votes_side.j__clarence_thomas,votes_side.j__ruth_bader_ginsburg,votes_side.j__stephen_g_breyer,votes_side.j__john_g_roberts_jr,votes_side.j__samuel_a_alito_jr,votes_side.j__sonia_sotomayor,votes_side.j__elena_kagan,votes_side.j__neil_gorsuch,votes_side.j__brett_m_kavanaugh
0,22620__0_000,We'll hear argument first this morning in 04-4...,j__john_g_roberts_jr,NaN,2005_04-433,J,2005,Anza v. Ideal Steel Supply Corporation,Roberts Court,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN
1,22620__0_002,"Well, isn't there something different here?\nB...",j__david_h_souter,22620__0_001,2005_04-433,J,2005,Anza v. Ideal Steel Supply Corporation,Roberts Court,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN
2,22620__0_004,"Sure, but they suffered the harm because the f...",j__david_h_souter,22620__0_003,2005_04-433,J,2005,Anza v. Ideal Steel Supply Corporation,Roberts Court,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN
3,22620__0_006,"Well, why... why is that true?\nLet's assume t...",j__anthony_m_kennedy,22620__0_005,2005_04-433,J,2005,Anza v. Ideal Steel Supply Corporation,Roberts Court,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN
4,22620__0_008,"Mr. Frederick, you... you started by saying ho...",j__ruth_bader_ginsburg,22620__0_007,2005_04-433,J,2005,Anza v. Ideal Steel Supply Corporation,Roberts Court,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125912,24969__2_003,-- what in reading this statute would give an ...,j__sonia_sotomayor,24969__2_002,2019_19-67,J,2019,United States v. Sineneng-Smith,Roberts Court,NaN,...,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
125913,24969__2_005,But accomplice liability --,j__sonia_sotomayor,24969__2_004,2019_19-67,J,2019,United States v. Sineneng-Smith,Roberts Court,NaN,...,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
125914,24969__2_007,-- has all sorts of meaning that you're not en...,j__sonia_sotomayor,24969__2_006,2019_19-67,J,2019,United States v. Sineneng-Smith,Roberts Court,NaN,...,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
125915,24969__2_009,-- altogether?,j__sonia_sotomayor,24969__2_008,2019_19-67,J,2019,United States v. Sineneng-Smith,Roberts Court,NaN,...,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
df_scotus['text'] = df_scotus['text'].str.replace('\n', ' ')
df_scotus['text']

0         We'll hear argument first this morning in 04-4...
1         Well, isn't there something different here? Be...
2         Sure, but they suffered the harm because the f...
3         Well, why... why is that true? Let's assume th...
4         Mr. Frederick, you... you started by saying ho...
                                ...                        
125912    -- what in reading this statute would give an ...
125913                          But accomplice liability --
125914    -- has all sorts of meaning that you're not en...
125915                                       -- altogether?
125916           Thank you, counsel. The case is submitted.
Name: text, Length: 125917, dtype: object

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [ ]:
# df_scotus['roberts_side'] = df_scotus['roberts_side'].astype(str).replace('2.0', 'majority')
# df_scotus['roberts_side'] = df_scotus['roberts_side'].astype(str).replace('-1.0', 'no vote')
# df_scotus['roberts_side'] = df_scotus['roberts_side'].astype(str).replace('1.0', 'minority')
df_roberts = df_scotus[df_scotus['speaker'] == 'j__john_g_roberts_jr']
df_roberts.columns

Index(['utt_id', 'text', 'speaker', 'reply_to', 'meta.case_id',
       'meta.speaker_type', 'year', 'title', 'court',
       'votes.j__john_paul_stevens', 'votes.j__sandra_day_oconnor',
       'votes.j__antonin_scalia', 'votes.j__anthony_m_kennedy',
       'votes.j__david_h_souter', 'votes.j__clarence_thomas',
       'votes.j__ruth_bader_ginsburg', 'votes.j__stephen_g_breyer',
       'votes.j__john_g_roberts_jr', 'votes.j__samuel_a_alito_jr',
       'votes.j__sonia_sotomayor', 'votes.j__elena_kagan',
       'votes.j__neil_gorsuch', 'votes.j__brett_m_kavanaugh',
       'votes_side.j__john_paul_stevens', 'votes_side.j__sandra_day_oconnor',
       'votes_side.j__antonin_scalia', 'votes_side.j__anthony_m_kennedy',
       'votes_side.j__david_h_souter', 'votes_side.j__clarence_thomas',
       'votes_side.j__ruth_bader_ginsburg', 'votes_side.j__stephen_g_breyer',
       'votes_side.j__john_g_roberts_jr', 'votes_side.j__samuel_a_alito_jr',
       'votes_side.j__sonia_sotomayor', 'votes_side.j

In [ ]:
from sklearn.svm import LinearSVC

# Predicting which way chief justice Roberts will vote
X_train = df_roberts[['text', 'votes_side.j__john_paul_stevens', 'votes_side.j__sandra_day_oconnor',
       'votes_side.j__antonin_scalia', 'votes_side.j__anthony_m_kennedy',
       'votes_side.j__david_h_souter', 'votes_side.j__clarence_thomas',
       'votes_side.j__ruth_bader_ginsburg', 'votes_side.j__stephen_g_breyer',
       'votes_side.j__samuel_a_alito_jr', 'votes_side.j__sonia_sotomayor',
       'votes_side.j__elena_kagan', 'votes_side.j__neil_gorsuch', 'votes_side.j__brett_m_kavanaugh']]
y_train = df_roberts['votes.j__john_g_roberts_jr']

transformer = make_column_transformer(
    (OneHotEncoder(handle_unknown="ignore"), ['votes_side.j__john_paul_stevens', 'votes_side.j__sandra_day_oconnor',
       'votes_side.j__antonin_scalia', 'votes_side.j__anthony_m_kennedy',
       'votes_side.j__david_h_souter', 'votes_side.j__clarence_thomas',
       'votes_side.j__ruth_bader_ginsburg', 'votes_side.j__stephen_g_breyer',
       'votes_side.j__samuel_a_alito_jr', 'votes_side.j__sonia_sotomayor',
       'votes_side.j__elena_kagan', 'votes_side.j__neil_gorsuch', 'votes_side.j__brett_m_kavanaugh']),
    (TfidfVectorizer(max_features=100), "text"),
    remainder="drop")

pipeline = make_pipeline(transformer, LinearSVC())
pipeline.fit(X_train, y_train)

scores = cross_val_score(pipeline, X_train, y_train, cv=10, scoring="accuracy", n_jobs=6)
print(f"Mean CV Accuracy: {scores.mean():.8f}")

Mean CV Accuracy: 0.87876641


In [ ]:
def get_best_model(pipeline, X_train, y_train):
  for n in range(3):
    grid_cv_linearSVC_1 = GridSearchCV(pipeline,
                          param_grid={
                              "linearsvc__C": [0.0001, 0.001, 0.01, 0.1, 1],
                          },
                          scoring="accuracy",
                          cv=10,
                          n_jobs=6)
    grid_cv_linearSVC_1.fit(X_train, y_train)
    pipeline.set_params(**grid_cv_linearSVC_1.best_params_)

    grid_cv_linearSVC_2 = GridSearchCV(pipeline,
                          param_grid={
                              "linearsvc__class_weight": ["balanced", None],
                          },
                          scoring="accuracy",
                          cv=10,
                          n_jobs=6)
    grid_cv_linearSVC_2.fit(X_train, y_train)
    pipeline.set_params(**grid_cv_linearSVC_2.best_params_)

    grid_cv_linearSVC_3 = GridSearchCV(pipeline,
                          param_grid={
                              "linearsvc__intercept_scaling": [0.5, 1, 2],
                          },
                          scoring="accuracy",
                          cv=10,
                          n_jobs=6)
    grid_cv_linearSVC_3.fit(X_train, y_train)
    pipeline.set_params(**grid_cv_linearSVC_3.best_params_)
    print(f'Iter {n+1} done', '\n')

  print(grid_cv_linearSVC_1.best_params_, grid_cv_linearSVC_2.best_params_, grid_cv_linearSVC_3.best_params_)
  return pipeline

pipeline = get_best_model(pipeline, X_train, y_train)

Iter 1 done 

Iter 2 done 

Iter 3 done 

{'linearsvc__C': 0.0001} {'linearsvc__class_weight': 'balanced'} {'linearsvc__intercept_scaling': 0.5}


In [ ]:
scores = cross_val_score(pipeline, X_train, y_train, cv=10, scoring="accuracy")
print(f"Mean CV Accuracy: {scores.mean():.8f}")

Mean CV Accuracy: 0.88068144


In [ ]:
justice_models = {}
justics_scores = {'justice': [],
                  'scores': []}
justices = df_scotus['speaker'].unique()
for justice in justices:
    df_justice = df_scotus[df_scotus['speaker'] == justice]

    columns = []
    for vJustice in justices:
        if vJustice != justice:
            columns.append(f'votes_side.{vJustice}')

    X_train = df_justice[['text'] + columns]
    y_train = df_justice[f'votes.{justice}'].fillna(-1)
    transformer = make_column_transformer(
        (OneHotEncoder(handle_unknown="ignore"), columns),
        (TfidfVectorizer(max_features=100), "text"),
        remainder="drop")

    pipeline = make_pipeline(transformer, LinearSVC())
    best_model = get_best_model(pipeline, X_train, y_train)

    scores = cross_val_score(best_model, X_train, y_train, cv=10, scoring="accuracy")
    print(f"Mean CV Accuracy for {justice}: {scores.mean():.8f}")
    justics_scores['justice'].append(justice)
    justics_scores['scores'].append(scores.mean())

    justice_models[justice] = best_model

Iter 1 done 

Iter 2 done 

Iter 3 done 

{'linearsvc__C': 0.001} {'linearsvc__class_weight': None} {'linearsvc__intercept_scaling': 0.5}
Mean CV Accuracy for j__john_g_roberts_jr: 0.88068144
Iter 1 done 

Iter 2 done 

Iter 3 done 

{'linearsvc__C': 1} {'linearsvc__class_weight': None} {'linearsvc__intercept_scaling': 1}
Mean CV Accuracy for j__david_h_souter: 0.69499837
Iter 1 done 

Iter 2 done 

Iter 3 done 

{'linearsvc__C': 0.001} {'linearsvc__class_weight': None} {'linearsvc__intercept_scaling': 0.5}
Mean CV Accuracy for j__anthony_m_kennedy: 0.90183501
Iter 1 done 

Iter 2 done 

Iter 3 done 

{'linearsvc__C': 0.001} {'linearsvc__class_weight': None} {'linearsvc__intercept_scaling': 0.5}
Mean CV Accuracy for j__ruth_bader_ginsburg: 0.77338909
Iter 1 done 

Iter 2 done 

Iter 3 done 

{'linearsvc__C': 0.001} {'linearsvc__class_weight': None} {'linearsvc__intercept_scaling': 1}
Mean CV Accuracy for j__john_paul_stevens: 0.75211830
Iter 1 done 

Iter 2 done 

Iter 3 done 

{'linea

In [ ]:
df_justice_scores = pd.DataFrame(justics_scores)
df_justice_scores

,justice,scores
0,j__john_g_roberts_jr,0.880681
1,j__david_h_souter,0.694998
2,j__anthony_m_kennedy,0.901835
3,j__ruth_bader_ginsburg,0.773389
4,j__john_paul_stevens,0.752118
5,j__samuel_a_alito_jr,0.762248
6,j__antonin_scalia,0.785604
7,j__stephen_g_breyer,0.775854
8,j__sandra_day_oconnor,0.713968
9,j__clarence_thomas,0.890909
